## Configure the CPDCTL for coneecting to a cluster

In [46]:
SPACE_NAME="p-test-space"
DEPLOY_NAME="p-test-deploy"

In [47]:
cpdctl config users list
cpdctl config users set qa-user --username=admin --password=password

Name      Username   
qa-user   admin   


In [48]:
cpdctl config profiles set qa-cluster --user qa-user --url https://zen-cpd-zen.apps.pwh.ocp.csplab.local

In [49]:
cpdctl config contexts set qa-context qa-cluster


In [50]:
str1=$(cpdctl spaces list | grep $SPACE_NAME | awk 'END{print NR}')
if [[ $str1 != 1 ]]; then
 echo "deleting extra spaces";
 cpdctl spaces list | grep $SPACE_NAME | awk 'NR>1{print $1}' | xargs -I{} cpdctl spaces delete --space_id {}
fi
if [[ $str1 == 0 ]]; then
 echo "creating space"
cpdctl spaces create --name $SPACE_NAME
 else
   cpdctl spaces list
   export SPACE_ID=$(cpdctl spaces list | grep $SPACE_NAME | awk '{ print $1}')
   echo $SPACE_ID
fi

...
ID                                     Name                  Created                    Tags   
713f8e3e-4a37-4961-8d08-c8cacfaf469d   p-test-space          2020-10-20T17:42:46.956Z   []   
8e00d316-c6f0-4705-9b90-8553bdbdf934   noah-test-space       2020-11-10T23:35:04.900Z   []   
95e3c6be-b06c-455b-9317-cfa9354943c3   approval-deployment   2020-10-28T20:37:31.341Z   []   
cda7fd92-0953-4758-95e6-39c75a2ae156   prod-space            2020-10-22T02:11:39.809Z   []   
fa6fb5f9-d676-421b-9e02-13bf4aa3c791   jenkinstestspace      2020-11-09T19:54:33.641Z   []   
713f8e3e-4a37-4961-8d08-c8cacfaf469d


In [51]:
models=$(cpdctl wml models list --space-id $SPACE_ID | awk 'NR>1')
if [[ $models == "Nothing to show." ]]; then
   echo "No models found, creating model"
   cpdctl wml models create --name mlopsmodel --space-id $SPACE_ID --software-spec '{"name": "scikit-learn_0.22-py3.6"}' --type 'scikit-learn_0.22'
   
   export MODEL_ID=$(cpdctl wml models list --space-id $SPACE_ID | grep mlopsmodel | awk '{ print $1}')
   cpdctl wml models upload-content --body mlopsmodelpipeline.tar.gz --model-id $MODEL_ID --content-format 'binary' --space-id $SPACE_ID --output json

 else
   cpdctl wml models list --space-id $SPACE_ID
   export MODEL_ID=$(cpdctl wml models list --space-id $SPACE_ID | grep mlopsmodel | awk '{ print $1}')
   echo $MODEL_ID
   cpdctl wml models upload-content --body mlopsmodelpipeline.tar.gz --model-id $MODEL_ID --content-format 'binary' --space-id $SPACE_ID --output json
   
fi

...
ID                                     Name         Created                    Type                Tags   
9401811f-8e3d-45d5-8540-b8266b7189e3   mlopsmodel   2020-10-20T18:00:19.002Z   scikit-learn_0.22   []   



In [52]:
deployments=$(cpdctl wml deployments list --space-id $SPACE_ID |awk 'NR>1')
echo $deployments
if [[ $deployments == "Nothing to show." ]]; then
echo "Creating Deployment"
cpdctl wml deployments create --name $DEPLOY_NAME --online '{"description": "mlopsdeploy"}' --asset '{"id": "'"$MODEL_ID"'"}' --space-id $SPACE_ID
fi
DEPLOY_ID=$(cpdctl wml deployments list --space-id $SPACE_ID | grep $DEPLOY_NAME| awk '{print $1}')
echo $DEPLOY_ID

ID Name Created Description State Tags 6f72017e-af84-4186-bc15-bf44ae5a3ff7 p-test-deploy 2020-10-20T18:00:40.501Z ready []



In [53]:
DEPLOY_URL=$(cpdctl wml deployments get --deployment-id $DEPLOY_ID --space-id $SPACE_ID --output yaml | grep https | awk '{print $2}')
echo $DEPLOY_URL

In [54]:
SERVICE_INSTANCE_ID=$(cpdctl wos service-providers list | awk 'NR>2 {print $1}')
echo $SERVICE_INSTANCE_ID

In [55]:
DATA_MART_ID=$(cpdctl wos data-marts list | awk 'NR>2{print $1}')
echo $DATA_MART_ID

In [56]:
cpdctl wml models list --space-id $SPACE_ID | awk 'NR==3{print $1}' | xargs -I{} cpdctl wml models get --model-id {} --space-id $SPACE_ID --output json

In [57]:

URL="https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v4/models/$MODEL_ID?space_id=$SPACE_ID"
echo $URL
ASSET='{"asset_id":"'"$MODEL_ID"'", "asset_type":"model" ,"problem_type":"multiclass", "input_data_type":"structured", "url":"'"$URL"'", "name":"'"$DEPLOY_NAME"'"}' 
DEPLOYMENT='{"deployment_id":"'"$DEPLOY_ID"'", "deployment_type":"online", "name":"'"$DEPLOY_NAME"'", "url": "'"$DEPLOY_URL"'", "scoring_endpoint":{ "request_headers": { "Content-Type": "application/json"}, "url":"'"$DEPLOY_URL"'"},     "service_provider_id": "fedb2dcd-1771-4705-88a1-b791a0bf5833"}'
ASSET_PROPERTIES=$(cat asset_properties.json)

In [58]:
echo $ASSET

{"asset_id":"9401811f-8e3d-45d5-8540-b8266b7189e3", "asset_type":"model" ,"problem_type":"multiclass", "input_data_type":"structured", "url":"https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v4/models/9401811f-8e3d-45d5-8540-b8266b7189e3?space_id=713f8e3e-4a37-4961-8d08-c8cacfaf469d", "name":"p-test-deploy"}


In [ ]:
#cpdctl wos subscriptions delete --subscription-id $SUBSCRIPTION_ID


In [ ]:
SUBSCRIPTION_ID=$(python -c "import json, sys; print([x.get('metadata').get('id') for x in json.loads(sys.argv[1]).get('subscriptions') if x.get('entity').get('deployment').get('name') ==sys.argv[2] ][0])" "$(cpdctl wos subscriptions list --output json)" $DEPLOY_NAME)
if [ -z "$SUBSCRIPTION_ID" ]
then 
    echo "Subscription not found"
    cpdctl wos subscriptions add --asset "$ASSET" --deployment "$DEPLOYMENT" --data-mart-id "$DATA_MART_ID" --service-provider-id "$SERVICE_INSTANCE_ID" --asset-properties "$ASSET_PROPERTIES"
    #python -c "import sys;from ibm_ai_openscale import APIClient4ICP, WatsonMachineLearningAsset; client = APIClient4ICP({'username':'admin', 'password':'password', 'url':'https://zen-cpd-zen.apps.pwh.ocp.csplab.local'});  print(client.data_mart.subscriptions.add(WatsonMachineLearningAsset(sys.argv[1], problem_type='multiclass', input_data_type'='structured, label_column='target'    ).get_details().get('metadata').get('id') ) " $MODEL_ID 
    sleep 10
    SUBSCRIPTION_ID=$(python -c "import json, sys; print([x.get('metadata').get('id') for x in json.loads(sys.argv[1]).get('subscriptions') if x.get('entity').get('deployment').get('name') ==sys.argv[2] ][0])" "$(cpdctl wos subscriptions list --output json)" $DEPLOY_NAME)
    sleep 10
    python -c "import sys; from ibm_ai_openscale import APIClient4ICP; client = APIClient4ICP({'username':'admin', 'password':'password', 'url':'https://zen-cpd-zen.apps.pwh.ocp.csplab.local'}); print(client.data_mart.subscriptions.get(name=sys.argv[1]).payload_logging.enable(dynamic_schema_update=True))"  $DEPLOY_NAME
else
    echo "Subscription found"
    echo $SUBSCRIPTION_ID
    echo "Deleting old subscription"
    cpdctl wos subscriptions delete --subscription-id $SUBSCRIPTION_ID
    cpdctl wos subscriptions add --asset "$ASSET" --deployment "$DEPLOYMENT" --data-mart-id "$DATA_MART_ID" --service-provider-id "$SERVICE_INSTANCE_ID" --asset-properties "$ASSET_PROPERTIES"
    sleep 10
    #python -c "import sys;from ibm_ai_openscale import APIClient4ICP, WatsonMachineLearningAsset; client = APIClient4ICP({'username':'admin', 'password':'password', 'url':'https://zen-cpd-zen.apps.pwh.ocp.csplab.local'});  print(client.data_mart.subscriptions.add(WatsonMachineLearningAsset(sys.argv[1]) ).get_details().get('metadata').get('id') ) " $MODEL_ID 
    SUBSCRIPTION_ID=$(python -c "import json, sys; print([x.get('metadata').get('id') for x in json.loads(sys.argv[1]).get('subscriptions') if x.get('entity').get('deployment').get('name') ==sys.argv[2] ][0])" "$(cpdctl wos subscriptions list --output json)" "mlopsmodel")
    python -c "import sys; from ibm_ai_openscale import APIClient4ICP; client = APIClient4ICP({'username':'admin', 'password':'password', 'url':'https://zen-cpd-zen.apps.pwh.ocp.csplab.local'}); print(client.data_mart.subscriptions.get(name=sys.argv[1]).payload_logging.enable(dynamic_schema_update=True))"  $DEPLOY_NAME

    #cpdctl wos subscriptions list
    sleep 5
fi
SUBSCRIPTION_ID=$(cpdctl wos subscriptions list | awk 'NR==3{print $1}')
echo $SUBSCRIPTION_ID


In [36]:
CONFIGURATIONS=$(cat configurations.json)
cpdctl wos subscriptions update --subscription-id $SUBSCRIPTION_ID --patch-document "[$CONFIGURATIONS]"
cpdctl wos subscriptions update --subscription-id $SUBSCRIPTION_ID --patch-document "[$ASSET_PROPERTIES]"


...
              
ID:        c2af5048-c957-4968-9898-6bb58c7d3a2f   
Created:   2020-11-11T00:20:35.527Z   
State:     active   
...
FAILED


              
metadata   <Nested Object>   
entity     <Nested Object>   


: 1

In [38]:
SUBSCRIPTION_ID=$(cpdctl wos subscriptions list | awk 'NR==3{print $1}')


In [41]:
cpdctl wos subscriptions list

...
ID                                     Created                    State   
e4cfe3c8-351a-4951-ab5e-548bcc024add   2020-11-11T00:42:06.866Z   active   
c25cd7ad-b98d-4b87-b862-3c27da4cef78   2020-11-10T23:35:34.061Z   active   
71966b72-11f9-4097-9571-6ddfeb966c92   2020-11-09T19:55:05.103Z   active   
52d91fa4-fa05-46d1-b736-d5e40f19d649   2020-11-03T20:57:12.023Z   active   
a867df0a-ace8-491d-8628-4b7737bcc63e   2020-11-03T16:21:59.202Z   active   


In [39]:
TOKEN=$(curl -k -X POST 'https://zen-cpd-zen.apps.pwh.ocp.csplab.local/icp4d-api/v1/authorize'  -d '{"username":"admin", "password":"password" }' --header 'Accept: application/json' --header "Content-Type: application/json" | grep token | tr -d "{}" | awk -F\: '{ print $4}' | tr -d \")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   817  100   773  100    44   2423    137 --:--:-- --:--:-- --:--:--  2561


In [40]:
#Check the newly created asset
curl  "https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v1/data_marts/$DATA_MART_ID/service_bindings/$SERVICE_INSTANCE_ID/subscriptions/$SUBSCRIPTION_ID/file" -k --header "authorization: Bearer $TOKEN" --header 'content-type: application/json'

{
  "asset": {
    "asset_id": "9401811f-8e3d-45d5-8540-b8266b7189e3",
    "asset_type": "model",
    "name": "p-test-deploy",
    "url": "https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v4/models/9401811f-8e3d-45d5-8540-b8266b7189e3?space_id=713f8e3e-4a37-4961-8d08-c8cacfaf469d"
  },
  "asset_properties": {
    "categorical_fields": [],
    "dashboard_configuration": {
      "monitor_preparation": {
        "completed": true
      }
    },
    "feature_fields": ["Column0", "Column1", "Column10", "Column11", "Column12", "Column13", "Column14", "Column15", "Column16", "Column17", "Column18", "Column19", "Column2", "Column20", "Column21", "Column22", "Column23", "Column24", "Column25", "Column26", "Column27", "Column28", "Column29", "Column3", "Column30", "Column31", "Column32", "Column33", "Column34", "Column35", "Column36", "Column37", "Column38", "Column39", "Column4", "Column40", "Column41", "Column42", "Column43", "Column44", "Column45", "Column46", "Column47", "Column48", "Column49",

In [44]:
cpdctl wos monitors instances add --data-mart-id $DATA_MART_ID --monitor-definition-id "quality" --parameters '{"min_feedback_data_size":1, "max_rows_per_evaluation":150}' --target '{"target_id":"'"$SUBSCRIPTION_ID"'", "target_type":"subscription"}'  --thresholds '[{"metric_id":"accuracy", "type":"lower_limit", "value":0.8}]'

...
              
ID:        cd0ee25e-191b-427a-9fdf-e91f2cc67ab4   
Created:   2020-11-11T00:50:45.656Z   
State:     preparing   


In [27]:
FIELDS=$(head -n1  data.csv | awk -F"," '{ $65=$66=$67="";print $0}'  | rev | cut -c 2- | rev |  awk -v OFS="," '{for (i=1; i<=NF; ++i){$i="\""$i"\""}}1')
for row in $(awk -F"," -v OFS="," 'NR>1200{$66=$65=""; print}' datawoprob.csv | rev| cut -c 3- | rev); 
do payload='[{"fields": '"[$FIELDS] "', "values": '"[[$row]]"'}]';
#echo $payload;
#echo $row;
response=$(cpdctl wml  deployments compute-predictions --deployment-id $DEPLOY_ID --input-data "$payload" --output json)
response=$(python -c "import sys, json ; print( json.dumps(json.loads(sys.argv[1]).get('predictions')[0]) )" "$response")
#echo $response
SCORING_PAYLOAD='[{
  "request": {
    "fields": '[$FIELDS]',
    "values": '[[$row]]'
  },
  "response": '$response',
  "binding_id": "'"$SERVICE_INSTANCE_ID"'",
  "subscription_id": "'"$SUBSCRIPTION_ID"'",
  "deployment_id": "'"$DEPLOY_ID"'"
}]'
echo $SCORING_PAYLOAD
curl -k -X POST https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v1/data_marts/00000000-0000-0000-0000-000000000000/scoring_payloads -d "$SCORING_PAYLOAD" --header 'Content-Type: application/json' --header 'Accept: application/json' --header "Authorization: Bearer $TOKEN"

done

[{ "request": { "fields": ["Column0","Column1","Column2","Column3","Column4","Column5","Column6","Column7","Column8","Column9","Column10","Column11","Column12","Column13","Column14","Column15","Column16","Column17","Column18","Column19","Column20","Column21","Column22","Column23","Column24","Column25","Column26","Column27","Column28","Column29","Column30","Column31","Column32","Column33","Column34","Column35","Column36","Column37","Column38","Column39","Column40","Column41","Column42","Column43","Column44","Column45","Column46","Column47","Column48","Column49","Column50","Column51","Column52","Column53","Column54","Column55","Column56","Column57","Column58","Column59","Column60","Column61","Column62","Column63"], "values": [[0.0,0.0,7.0,16.0,13.0,4.0,0.0,0.0,0.0,0.0,7.0,16.0,16.0,9.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,9.0,0.0,0.0,0.0,0.0,3.0,16.0,16.0,9.0,0.0,0.0,0.0,0.0,8.0,16.0,16.0,8.0,0.0,0.0,0.0,0.0,10.0,16.0,16.0,4.0,0.0,0.0,0.0,0.0,9.0,16.0,14.0,1.0,0.0,0.0,0.0,0.0,4.0,12.0,14.0,8.0,

In [42]:
VALUES=$(awk -F"," -v OFS="," 'NR>1000{ print $0  }' datawoprob.csv | rev| cut -c 3- | rev | awk -F"," -v OFS="," '{printf t "["$0"]"}{t=", "}' )


In [45]:
python -c "import sys,json; from ibm_ai_openscale import APIClient4ICP; APIClient4ICP({'username':'admin', 'password':'password', 'url':' https://zen-cpd-zen.apps.pwh.ocp.csplab.local' }).data_mart.subscriptions.get(name=sys.argv[1]).feedback_logging.store(json.loads(sys.argv[2]).get('values') ) " $DEPLOY_NAME '{"values":'"[$VALUES]"'}'

/usr/local/Caskroom/miniconda/base/envs/pyspark/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.apps.pwh.ocp.csplab.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/pyspark/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.apps.pwh.ocp.csplab.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/pyspark/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.apps.pwh.ocp.csplab.local'. Adding certificate verification is strongly a

In [ ]:
VALUES=$(awk -F"," -v OFS="," 'NR>1000{$66=$65=""; print $0  }' datawoprob.csv | rev| cut -c 3- | rev | awk -F"," -v OFS="," '{printf t "["$0"]"}{t=", "}' )
FIELDS=$(head -n1  data.csv | awk -F"," '{ $65=$66=$67="";print $0}'  | rev | cut -c 2- | rev |  awk -v OFS="," '{for (i=1; i<=NF; ++i){$i="\""$i"\""}}1')
payload='[{"fields": '"[$FIELDS]"', "values": '"[$VALUES]"' }]'
echo $payload
cpdctl wml  deployments compute-predictions --deployment-id $DEPLOY_ID --input-data "$payload" --output json


In [ ]:
FIELDS=$(head -n1  data.csv | awk -F"," '{ $65=$66=$67="";print $0}'  | rev | cut -c 2- | rev |  awk -v OFS="," '{for (i=1; i<=NF; ++i){$i="\""$i"\""}}1')
for row in $(awk -F"," -v OFS="," 'NR>1{$66=$65=""; print}' datawoprob.csv | rev| cut -c 3- | rev); 
do payload='[{"fields": '"[$FIELDS] "', "values": '"[[$row]]"'}]';
echo $payload;
#python -c "import sys, json ; print(json.loads(sys.argv[1]).get('response'))" $(cpdctl wml  deployments compute-predictions --deployment-id $DEPLOY_ID --input-data "$payload" --output json)
#curl -k -X POST https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v1/data_marts/00000000-0000-0000-0000-000000000000/scoring_payloads -d "$SCORING_PAYLOAD" \ --header 'Content-Type: application/json' --header 'Accept: application/json' --header "Authorization: Bearer $ICP_TOKEN"

# done

In [415]:
MONITOR_INSTANCE_ID=$(cpdctl wos monitors instances list | awk 'NR==3{print $1}')
echo $MONITOR_INSTANCE_ID

422774fd-8b88-4b8b-b7f1-3d6db01cb2b8


In [428]:
cpdctl wos monitors instances list

...
ID                                     Created                    State   
422774fd-8b88-4b8b-b7f1-3d6db01cb2b8   2020-10-30T22:01:13.539Z   active   
a69115ff-f3fb-42cc-bd1a-b1531f5b45a8   2020-10-30T19:15:28.830Z   active   
5e9db4fb-d6f8-44ee-bcd5-efd17e67e8b8   2020-10-30T19:07:21.267Z   active   


In [429]:
#Check that the monitor is active
cpdctl wos monitors instances get --monitor-instance-id $MONITOR_INSTANCE_ID

...
              
ID:        422774fd-8b88-4b8b-b7f1-3d6db01cb2b8   
Created:   2020-10-30T22:01:13.539Z   
State:     active   


In [430]:
cpdctl wos monitors runs add --monitor-instance-id $MONITOR_INSTANCE_ID

...
FAILED
Error processing the response received, most likely an error response:
Post "https://zen-cpd-zen.apps.pwh.ocp.csplab.local/openscale/00000000-0000-0000-0000-000000000000/v2/monitor_instances/422774fd-8b88-4b8b-b7f1-3d6db01cb2b8/runs": context deadline exceeded (Client.Timeout exceeded while awaiting headers)



: 1

In [433]:
cpdctl wos monitors runs list --monitor-instance-id $MONITOR_INSTANCE_ID

...
ID                                     Created                    State   
617745fc-66f5-4616-b828-6f4af3540b12   2020-10-30T22:30:03.363Z   finished   
e218ed63-cc52-408c-8826-39368505f528   2020-10-30T22:24:21.649Z   error   
c3aacf42-b4c8-4dc5-88c9-25380c5a57de   2020-10-30T22:02:08.220Z   error   


In [440]:
cpdctl wos monitors runs get --monitoring-run-id $(cpdctl wos monitors runs list --monitor-instance-id $MONITOR_INSTANCE_ID | awk 'NR==3{print $1}')  --monitor-instance-id $MONITOR_INSTANCE_ID --output yaml

entity:
  parameters:
    max_rows_per_evaluation: 150
    min_feedback_data_size: 1
    total_records_processed: 798
  status:
    completed_at: "2020-10-30T22:31:30.406Z"
    operators:
    - id: original
      result:
        confusion_matrix:
          labels:
          - "0"
          - "1"
          - "2"
          - "3"
          - "4"
          - "5"
          - "6"
          - "7"
          - "8"
          - "9"
          metrics_per_label:
            false_positive_rate_per_label:
            - 0
            - 0
            - 0
            - 0
            - 0
            - 0
            - 0
            - 0
            - 0
            - 0
            precision_per_label:
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
            recall_per_label:
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
            - 1
        